# Soulution

In [1]:
import sys
from simulator import simulate_revenue, score_me
import numpy as np
from scipy.optimize import brentq, fmin_cobyla

In [2]:
def optimization(Bar_For_Flight, Ave):
    if Bar_For_Flight <= 0:
        return np.zeros_like(Ave[:, :1])
    lin = 0.5 * Ave / Bar_For_Flight
    if lin.ndim < 2:
        lin = lin.reshape(1, -1)
    G_Func = lambda C, z: 1 - np.sum(np.maximum(z - C, 0))
    c_optimize = np.array([[brentq(G_Func, 0, max(max(x), 0), args=(x,))
                       if G_Func(0, x) < 0 else 0 for x in lin]]).T
    x_optimize = np.maximum(lin - c_optimize, 0)
    v_optimize = c_optimize + 0.5 * np.sum(x_optimize**2, axis=-1, keepdims=True)
    return 2 * Bar_For_Flight * Bar_For_Flight * v_optimize

In [3]:
class Basement(object):
    def __init__(self):
        pass
    
    def __call__(self, days_left, tickets_left, demand_level):
        return demand_level - 10

In [4]:
class Optimization_Policy(Basement):
    def __init__(self, demand_bins):
        super(Optimization_Policy, self).__init__()
        self.demand_bins = demand_bins

    def __call__(self, days_left, tickets_left, demand_level):
        tickets_left, days_left = int(tickets_left), int(days_left)
        self.computation(days_left, tickets_left)

        demand_level_bin = np.digitize(demand_level, self.demand_bins) - 1

        quene = np.argmax(self.current_[demand_level_bin, :tickets_left + 1]
                        + self.value_[days_left - 1, tickets_left::-1])

        price = demand_level - quene
        return price

    def computation(self, days_left, tickets_left):
        dp_computed_, tickets_left = hasattr(self, "value_"), int(tickets_left)
        if dp_computed_:
            n_days, n_tickets_p1 = self.value_.shape
            dp_computed_ = (n_days >= days_left) and (n_tickets_p1 > tickets_left)
        if dp_computed_:
            return
        
        self.value_, self.current_ = self._computation(days_left, tickets_left)
    
    def _computation(self, n_days, n_tickets):
        current = np.zeros((len(self.demand_bins), 1 + n_tickets), dtype=float)
        for q in range(1 + n_tickets):
            current[:, q] = (self.demand_bins - q) * q
            
        V_tilde = np.zeros((n_days, 1 + n_tickets), dtype=float)
        for t in range(1, n_days):
            for x in range(1 + n_tickets):
                V_txq = current[:, :x + 1] + V_tilde[t - 1, np.newaxis, x::-1]
                V_tilde[t, x] = np.mean(np.max(V_txq, axis=-1), axis=0)
        return V_tilde, current

In [5]:
pricing_function = Optimization_Policy(np.linspace(100, 200, num=2001))

In [6]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 136. Price set to $136. Sold 0 tickets. 
Daily revenue is 0. Total revenue-to-date is 0. 50 seats remaining

6 days before flight: Started with 50 seats. Demand level: 159. Price set to $149. Sold 10 tickets. 
Daily revenue is 1490. Total revenue-to-date is 1490. 40 seats remaining

5 days before flight: Started with 40 seats. Demand level: 188. Price set to $168. Sold 20 tickets. 
Daily revenue is 3353. Total revenue-to-date is 4843. 20 seats remaining

4 days before flight: Started with 20 seats. Demand level: 196. Price set to $179. Sold 17 tickets. 
Daily revenue is 3044. Total revenue-to-date is 7887. 3 seats remaining

3 days before flight: Started with 3 seats. Demand level: 161. Price set to $160. Sold 1 tickets. 
Daily revenue is 160. Total revenue-to-date is 8047. 2 seats remaining

2 days before flight: Started with 2 seats. Demand level: 189. Price set to $187. Sold 2 tickets. 
Daily revenue is 375. Total revenue-to

8421.066465326696